# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- SQL and Relational Databases
- Object Oriented Programming
- Accessing Data Through APIs
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In [1]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3

## Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [ ]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

1. For any **one** Pokemon, retrive the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    
For `Types` and `Abilities`, you will want to write helper functions to have each attribute just be a list of types and a list of abilities.

- i.e. `bulbasaur.types` should return `['poison', 'grass']`
    
    
2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the dictionary. (Hint: Use pagination)

In [ ]:
""" SOLUTION: data for one Pokemon """

# previewing the result for one pokemon

requests.get(results[0]['url']).json()


In [ ]:
def typelist(types):
    result = []
    
    # iterating through the nested dict and appending to the empty list:
    
    for i in range(len(types)):
        result.append(types[i]['type']['name'])
    return result

In [ ]:
def abilitylist(abilities):
    result = []
    
    # iterating through the nested dict and appending to the empty list:
    
    for i in range(len(abilities)):
        result.append(abilities[i]['ability']['name'])
    return result

In [ ]:
def get_pokedata(url):
    
    # getting full results for one pokemon
    info = requests.get(url).json() 
    
    # list of keys with values that don't need editing
    keys = ['id', 'name', 'base_experience', 'weight', 'height'] 
    data = {k: info[k] for k in keys} # dictionary comprehension
    
    # using the two helper functions to add types and abilities
    data['types'] = typelist(info['types'])
    data['abilities'] = abilitylist(info['abilities'])
    
    return data

In [ ]:


get_pokedata('https://pokeapi.co/api/v2/pokemon/1/')

In [ ]:
""" SOLUTION: data for 151 Pokemon """

# list comprehension to get a list of just URLs
urls = [r['url'] for r in results]

# list comprehension with the previous function to get full data
pokedata = [get_pokedata(url) for url in urls]


In [ ]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

## Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    
    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.


In [ ]:
# if you were unable to get the data from the API in the right format,
# this JSON file has the list of dictionaries

with open('data/pokemon.json') as f:  
    pokelist = json.load(f)

In [ ]:
class Pokemon:
    
    def __init__(self, ID, name, exp, weight, height, types, abilities):
        self.ID = ID
        self.name = name
        self.exp = exp
        self.weight = weight
        self.height = height
        self.types = types
        self.abilities = abilities
        
    def bmi(self):
        return (self.weight*0.1)/(self.height*0.1)**2
        


In [ ]:
""" SOLUTION: instantiating three Pokemon """

# function to instantiate a Pokemon
def instantiate_pokemon(info):
    pokemon = Pokemon(info['id'], 
                      info['name'], 
                      info['base_experience'], 
                      info['weight'], 
                      info['height'], 
                      info['types'],
                      info['abilities'])
    return pokemon

"""
can also be done manually:

bulbasaur = Pokemon(1, 'bulbasaur', 64, 69, 7, ['poison', 'grass'], ['chlorophyll', 'overgrow'])

etc.

"""

In [ ]:
bulbasaur = instantiate_pokemon(pokedata[0])
ivysaur = instantiate_pokemon(pokedata[3])
venusaur = instantiate_pokemon(pokedata[6])

In [ ]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.exp) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(ivysaur)
print_pokeinfo(venusaur)

### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [ ]:
# run this cell to test and check your code
# you probably have to rerun the code to instantiate your objects

bulbasaur = Pokemon.all_pokemon[0]

print("{}'s BMI is {}.".format(bulbasaur.name.title(), bulbasaur.bmi())) # 14.08

## SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

(insert dbdiagram)

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types


In [2]:
cnx = sqlite3.connect('data/pokemon.db')

In [ ]:
q1 = "select * from pokemon where base_experience > 150"
pd.read_sql(q1, cnx)

In [ ]:
q2 = "select id, name, type1, type2 from pokemon where type1 == 'water' or type2 == 'water'"
pd.read_sql(q2, cnx)

In [ ]:
q3 = "select * from pokemon join types"
pd.read_sql(q3, cnx)